# Main Notebook

In [1]:
import re
import os
import spacy
nlp = spacy.load("en_core_web_sm")

## Parse Data

### coca-samples-text

In [2]:
def parse_articles_text(file_text): 
    article_texts = file_text.split("\n")

    articles = []
    for article_text in article_texts: 
        if len(article_text) == 0: 
            continue
        
        pattern = r'@@\d+ '
        rv = re.findall(pattern, article_text[:20])
        if len(rv) == 0: 
            continue
        article_number = int(rv[0][2:-1])
        print(f"article_number: {article_number}")
        article_doc = nlp(article_text)
        article = {"number" : article_number, "doc" : article_doc}
        
        articles.append(article)
    
    if len(articles) == 0: 
        return None
    return articles

In [3]:
def parse_filename_text(directory, filename): 
    print(f"filename: {filename}")
    file = open(directory + filename, "r", encoding="ISO-8859-1")
    file_text = file.read()
    file.close()
    
    articles = parse_articles_text(file_text)
    if articles == None: 
        return None
    file = {"filename" : filename, "articles" : articles}
    return file

In [4]:
def get_files_text(directory): 
    files = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        
        file = parse_filename_text(directory, filename)
        if file == None: 
            continue
        files.append(file)
    return files

## Build Data

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
%%time
text_directory = "data/coca-samples-text/"
# text_files = get_files_text(text_directory)

# THIS WAS TAKING TOO LONG SO I AM ONLY DOING SPOK AND TVM
# ['text_news.txt', 'text_fic.txt', 'text_web.txt', 'text_spok.txt', 'text_tvm.txt', 
# 'text_blog.txt', 'text_acad.txt', 'text_mag.txt']
text_files = [parse_filename_text(text_directory, 'text_spok.txt'), parse_filename_text(text_directory, 'text_tvm.txt')]

filename: text_spok.txt
article_number: 17141
article_number: 21741
article_number: 207541
article_number: 207641
article_number: 220641
article_number: 220741
article_number: 221141
article_number: 221241
article_number: 221341
article_number: 221441
article_number: 221541
article_number: 221641
article_number: 221741
article_number: 222141
article_number: 222241
article_number: 222341
article_number: 222441
article_number: 222541
article_number: 222741
article_number: 222841
article_number: 222941
article_number: 223041
article_number: 223141
article_number: 223241
article_number: 223341
article_number: 223441
article_number: 223541
article_number: 223941
article_number: 224241
article_number: 224341
article_number: 224541
article_number: 224641
article_number: 224741
article_number: 224841
article_number: 224941
article_number: 225041
article_number: 225141
article_number: 225241
article_number: 225341
article_number: 225441
article_number: 225541
article_number: 225641
article_numb

article_number: 5225341
article_number: 5225441
article_number: 5225841
article_number: 5226041
article_number: 5226141
article_number: 5226241
article_number: 5226341
article_number: 5226441
article_number: 5226541
article_number: 5226641
article_number: 5226741
article_number: 5227241
article_number: 5227341
article_number: 5227441
article_number: 5227541
article_number: 5227941
article_number: 5228041
article_number: 5228141
article_number: 5228241
article_number: 5229041
article_number: 5229141
article_number: 5229241
article_number: 5229441
article_number: 5229641
article_number: 5230041
article_number: 5230141
article_number: 5230341
article_number: 5230441
article_number: 5231141
article_number: 5231241
article_number: 5231341
article_number: 5231541
article_number: 5231641
article_number: 5231741
article_number: 5232141
article_number: 5232241
article_number: 5232341
article_number: 5232541
article_number: 5232641
article_number: 5233041
article_number: 5233141
article_number: 

In [7]:
def build_data(label1, label2): 
    text_file1 = [file for file in text_files if label1 in file["filename"]][0]
    text_file2 = [file for file in text_files if label2 in file["filename"]][0]
    
    X, Y = [], []
    for article in text_file1["articles"]: 
        X.append(article)
        Y.append(label1)
    
    for article in text_file2["articles"]: 
        X.append(article)
        Y.append(label2)
    return (X, Y)

In [8]:
X, Y = build_data("spok", "tvm")
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=0)

## Build Classifier

In [9]:
import json
import nltk
from sklearn import linear_model
from scipy import sparse
from collections import Counter
import operator

In [10]:
def print_weights(clf, vocab, n=10):
    weights=clf.coef_[0]
    reverse_vocab=[None]*len(weights)
    for k in vocab:
        reverse_vocab[vocab[k]]=k

    print(f"Class 1: {clf.classes_[0]}")
    for feature, weight in sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))[:n]:
        print("%.3f\t%s" % (weight, feature))

    print()

    print(f"Class 2: {clf.classes_[1]}")
    for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
        print("%.3f\t%s" % (weight, feature))

In [11]:
def build_features(dataX, feature_functions):
    
    """ This function featurizes the data according to the list of parameter feature_functions """
    
    data=[]
    for tokens in dataX:
        feats={}
        
        for function in feature_functions:
            feats.update(function(tokens))

        data.append(feats)
    return data

In [12]:
def features_to_ids(data, feature_vocab):
    
    """ 
    
    This helper function converts a dictionary of feature names to a sparse representation
 that we can fit in a scikit-learn model.  This is important because almost all feature 
 values will be 0 for most documents (note: why?), and we don't want to save them all in 
 memory.

    """
    new_data=sparse.lil_matrix((len(data), len(feature_vocab)))
    for idx,doc in enumerate(data):
        for f in doc:
            if f in feature_vocab:
                new_data[idx,feature_vocab[f]]=doc[f]
    return new_data

In [13]:
def create_vocab(data, top_n=None):
    
    """ 
    
    This helper function converts a dictionary of feature names to unique numerical ids. 
    top_n limits the features to only the n most frequent features observed in the training data 
    (in terms of the number of documents that contains it).
    
    """
    
    counts=Counter()
    for doc in data:
        for feat in doc:
            counts[feat]+=1

    feature_vocab={}

    for idx, (k, v) in enumerate(counts.most_common(top_n)):
        feature_vocab[k]=idx
                
    return feature_vocab

In [14]:
def pipeline(trainX, devX, trainY, devY, feature_functions):

    """ This function evaluates a list of feature functions on the training/dev data arguments """
    
    trainX_feat=build_features(trainX, feature_functions)
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data.
    feature_vocab=create_vocab(trainX_feat, top_n=100000)

    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    clf = linear_model.LogisticRegression(C=100, solver='lbfgs', penalty='l2', max_iter=10000)
    clf.fit(trainX_ids, trainY)
    print("Accuracy: %.3f" % clf.score(devX_ids, devY))
    
    return clf, feature_vocab

## Build Features

In [15]:
def bag_of_words(article):
    doc = article["doc"]
    
    feats = {}
    for token in doc: 
        feat_name = f"token_{token.text}"
        feats[feat_name] = feats.get(feat_name, 0) + 1
    return feats

In [16]:
def sentence_length(article): 
    doc = article["doc"]
    
    feats = {}
    for sentence in list(doc.sents): 
        num_tokens = len(list(doc.sents))
        feat_name = f"sentence_length_{num_tokens}"
        feats[feat_name] = feats.get(feat_name, 0) + 1
    return feats

In [17]:
def parts_of_speech(article): 
    doc = article["doc"]
    
    feats = {}
    for token in doc: 
        feat_name = f"POS_{token.pos_}"
        feats[feat_name] = feats.get(feat_name, 0) + 1
    return feats

In [18]:
hedging_file = open("data/hedging_data.txt")
hedging_text = hedging_file.read()
hedging_file.close()
hedging_lst = [h for h in hedging_text.split("\n") if ("%" not in h and len(h) > 2)]
print(hedging_lst)

['largely', 'generally', 'often', 'rarely', 'sometimes', 'frequently', 'occasionally', 'seldom', 'usually', 'most', 'several', 'some', 'almost', 'practically', 'apparently', 'virtually', 'basically', 'approximately', 'roughly', 'somewhat', 'somehow', 'partially', 'actually', 'like', 'something', 'someone', 'somebody', 'somewhere', 'think', 'thinks', 'thought', 'believe', 'believed', 'believes', 'consider', 'considers', 'considered', 'assume', 'assumes', 'assumed', 'understand', 'understands', 'understood', 'find', 'found', 'finds', 'appear', 'appears', 'appeared', 'seem', 'seems', 'seemed', 'suppose', 'supposes', 'supposed', 'guess', 'guesses', 'guessed', 'estimate', 'estimates', 'estimated', 'speculate', 'speculates', 'speculated', 'suggest', 'suggests', 'suggested', 'may', 'could', 'should', 'might', 'surely', 'probably', 'likely', 'maybe', 'perhaps', 'unsure', 'probable', 'unlikely', 'possibly', 'possible', 'read', 'say', 'says', 'looks like', 'look like', "don't know", 'necessarily

In [19]:
def hedging_feature(article): 
    doc = article["doc"]
    
    feats = {}
    for sentence in list(doc.sents): 
        for hedge in hedging_lst: 
            if hedge in sentence.string: 
                feat_name = f"hedge_{hedge}"
                feats[feat_name] = feats.get(feat_name, 0) + 1
    return feats

## Put It All Together

In [20]:
features = [bag_of_words, sentence_length, parts_of_speech, hedging_feature]

In [ ]:
clf, vocab = pipeline(trainX, testX, trainY, testY, features)

In [ ]:
print_weights(clf, vocab, n=20)

## Parse Litbank Data

In [ ]:
# nlp.max_length = 2000000

In [ ]:
litbank_dir = "../litbank/original/"

In [ ]:
def parse_text_litbank(number, text): 
    doc = nlp(text)
    return {"number" : number, "doc" : doc}

In [ ]:
def build_litbank_data(): 
    X = []
    number = 1
    for filename in os.listdir(litbank_dir)[:3]:
        if not filename.endswith(".txt"): 
            continue
        full_filename = os.path.join(litbank_dir, filename)
        file = open(full_filename)
        text = file.read()
        X.append(parse_text_litbank(number, text))
        file.close()

        number += 1
    
    return X

In [ ]:
%%time
X_litbank = build_litbank_data()

In [ ]:
def predict_data(X): 
    X_feat = build_features(X, features)
    X_ids=features_to_ids(X_feat, vocab)
    return clf.predict_proba(X_ids)

In [ ]:
p = predict_data(X_litbank)
print(p)